# Clean

Notebook clean functionality ported from
[nbdev clean](https://github.com/AnswerDotAI/nbdev/blob/main/nbs/api/11_clean.ipynb)
by jdawg

In [ ]:
// | export

import type { Cell } from "jurassic/notebooks.ts";
import { loadNb, saveNb } from "jurassic/notebooks.ts";

In [ ]:
//| export

const reprIdRegex = /(<.*?)( at 0x[0-9a-fA-F]+)(>)/;

const skipOrSub = (x: string): string =>
  x.includes("at 0x") ? x.replace(reprIdRegex, "$1$3") : x;

const cleanCellOutputId = (
  lines: string | string[],
): string | string[] =>
  typeof lines === "string" ? skipOrSub(lines) : lines.map((o) => skipOrSub(o));

In [ ]:
import { assertEquals } from "jsr:@std/assert";

Deno.test("cleanCellOutputId - array input", () => {
  assertEquals(
    cleanCellOutputId([
      "Lambda(func=)",
      "[,\n",
      "(, , )",
    ]),
    [
      "Lambda(func=)",
      "[,\n",
      "(, , )",
    ],
  );
});

Deno.test("cleanCellOutputId - string input", () => {
  assertEquals(
    cleanCellOutputId("foo\n\nbar"),
    "foo\n\nbar",
  );
});

In [ ]:
//| export

const cleanCellOutput = (cell: Cell, cleanIds: boolean): Cell => {
  const outputs = cell.outputs || [];

  outputs.forEach((o) => {
    if ("execution_count" in o) {
      o.execution_count = null;
    }

    const data = o.data || {};
    delete data["application/vnd.google.colaboratory.intrinsic+json"];

    for (const k in data) {
      if (k.startsWith("text") && cleanIds) {
        data[k] = cleanCellOutputId(data[k]);
      }
      if (k.startsWith("image") && !k.includes("svg")) {
        data[k] = data[k].trimEnd();
      }
    }

    if (o.text && cleanIds) {
      o.text = cleanCellOutputId(o.text);
    }

    if (o.metadata) {
      delete o.metadata.tags;
    }
  });

  return cell;
};

In [ ]:
//| export

const cleanCell = (
  cell: Cell,
  clearAll: boolean,
  allowedMetadataKeys: string[],
  cleanIds: boolean,
): Cell => {
  if ("execution_count" in cell) {
    cell.execution_count = null;
  }

  if ("outputs" in cell) {
    if (clearAll) {
      cell.outputs = [];
    } else {
      cleanCellOutput(cell, cleanIds);
    }
  }

  if (cell.source && cell.source.length === 1 && cell.source[0] === "") {
    cell.source = [];
  }

  cell.metadata = clearAll ? {} : Object.fromEntries(
    Object.entries(cell.metadata || {})
      .filter(([k]) => allowedMetadataKeys.includes(k)),
  );

  return cell;
};

In [ ]:
//| export

export const cleanNotebook = async (
  nbPath: string,
  clearAll = false,
  allowedMetadataKeys: string[] = [],
  allowedCellMetadataKeys: string[] = [],
  cleanIds = true,
): Promise<void> => {
  const nb = await loadNb(nbPath);
  const metadataKeys = new Set([
    "kernelspec",
    "jekyll",
    "jupytext",
    "doc",
    "widgets",
    ...allowedMetadataKeys,
  ]);

  const cellMetadataKeys = new Set([
    "hide_input",
    ...allowedCellMetadataKeys,
  ]);

  nb.cells.forEach((c) =>
    cleanCell(c, clearAll, Array.from(cellMetadataKeys), cleanIds)
  );

  const kernelName = nb.metadata?.kernelspec?.name;
  if (nb.metadata && kernelName) {
    nb.metadata.kernelspec.display_name = kernelName;
  }

  nb.metadata = Object.fromEntries(
    Object.entries(nb.metadata || {})
      .filter(([k]) => metadataKeys.has(k)),
  );

  // additional cleaning
  nb.cells = nb.cells.map((c: Cell) => {
    if (
      c.cell_type !== "code" ||
      !c.source.join("\n").includes("Deno.test")
    ) {
      return c;
    }
    return Object.assign({}, c, {
      outputs: [],
    });
  });

  await saveNb(nb);
};

In [ ]:
// import { j } from "jurassic/jdawg.ts";

// j.initialize("clean.ipynb");

// await j`convert this python to typescript:
// def clean_nb(
//     nb, # The notebook to clean
//     clear_all=False, # Remove all cell metadata and cell outputs?
//     allowed_metadata_keys:list=None, # Preserve the list of keys in the main notebook metadata
//     allowed_cell_metadata_keys:list=None, # Preserve the list of keys in cell level metadata
//     clean_ids=True, # Remove ids from plaintext reprs?
// ):
//     "Clean nb from superfluous metadata"
//     metadata_keys = {"kernelspec", "jekyll", "jupytext", "doc", "widgets"}
//     if allowed_metadata_keys: metadata_keys.update(allowed_metadata_keys)
//     cell_metadata_keys = {"hide_input"}
//     if allowed_cell_metadata_keys: cell_metadata_keys.update(allowed_cell_metadata_keys)
//     for c in nb['cells']: _clean_cell(c, clear_all, cell_metadata_keys, clean_ids)
//     if nb.get('metadata', {}).get('kernelspec', {}).get('name', None):
//         nb['metadata']['kernelspec']['display_name'] = nb["metadata"]["kernelspec"]["name"]
//     nb['metadata'] = {k:v for k,v in nb['metadata'].items() if k in metadata_keys}
// `;